In [1]:
import time
from pyspark.sql import SparkSession
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = 30
pd.options.display.max_colwidth = 150

spark = SparkSession \
    .builder \
    .appName("Session-10") \
    .getOrCreate()

from pyspark import SparkConf
conf = SparkConf().setMaster("local[2]").setAppName("Session_10")\
.set("some.conf", "to.some.value")

In [2]:
import tweepy
import json
from pathlib import Path

ACCESS_TOKEN = "YOUR OWN TOKEN"
ACCESS_SECRET =  "YOUR OWN TOKEN"
CONSUMER_KEY =  "YOUR OWN TOKEN"
CONSUMER_SECRET =  "YOUR OWN TOKEN"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

In [3]:
public_tweets = api.home_timeline()
for tweet in public_tweets[:5]:
    print(tweet.text)
    

Pune-based @h2epower is developing India’s first totally integrated #hydrogen #fuel cell three-wheeler using PEM fu… https://t.co/15v6CnCudG
RT @thequote: There's no shortcut to a place worth going. - Beverly Sills
Business As Usual by E P Unny. 
For more click here: https://t.co/KIuUZguxLB https://t.co/VkeEBEezIQ
RT @COVIDNewsByMIB: #IndiaFightsCorona:

📍𝗡𝗲𝘄 𝗥𝗲𝗰𝗼𝘃𝗲𝗿𝗶𝗲𝘀 have exceeded 𝗡𝗲𝘄 𝗖𝗮𝘀𝗲𝘀 for the last 25 days 

☑️Follow #COVIDAppropriateBehaviour…
RT @aapkadharam: Dosto, Dalip Sahab💕 ek nek rooh insaan...ek azeem fankaar ke liye aap ki rooh se uthi duaen zaroor bar aayengi 🙏🙏🙏🙏🙏🙏🙏 ji…


In [4]:
# Extract the information for a specific user
user = api.get_user('USAndIndia')
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
    print(friend.screen_name)

USAndIndia
209413
SandhuTaranjitS
SecBecerra
AmbassadorTai
PowerUSAID
Surgeon_General
DeputySecState
PressSec
WHNSC
USAmbUN
JakeSullivan46
ClimateEnvoy
SecDef
POTUS
VP
SecBlinken
WhiteHouse
USEmbassyFrance
AkashvaniAIR
DDIndialive
videovolunteers


In [ ]:
# Follow each and every follower that follows our account
for follower in tweepy.Cursor(api.followers).items():
    follower.follow()
    

In [5]:
usembindia = api.get_user('USAndIndia')
print(usembindia.screen_name)
print(usembindia.followers_count)
usemb_friends= []
for friend in usembindia.friends():
    usemb_friends.append(friend.screen_name)

USAndIndia
209413


In [10]:
for friend in usembindia.followers():
    print(friend.followers_count)

3
58
0
1
0
43
5
2
6
4
11
1
4
4
65
0
81
28
29
13


In [11]:
for person in api.search_users("Narendra Modi")[:5]:
    print(person.screen_name)

narendramodi
narendramodi_in
PMOIndia
NamoApp
pragnik


### Direct Messaging

In [ ]:
api.send_direct_message()

### Follower operations

In [ ]:
# to follow a given profile
api.create_friendship("TensorFlow")

In [ ]:
# to follow or unfollow a given profile
api.destroy_friendship("TensorFlow")

In [ ]:
api.followers_ids("TensorFlow")[:3]

In [ ]:
api.trends_available()

### Real Time Twitter Streaming

In [12]:
import json
from pathlib import Path

public_tweets = api.home_timeline()
for tweet in public_tweets:
    base= Path("twitter-logs")
    jsonpath = base / (tweet.id_str + ".json")
    base.mkdir(exist_ok=True)
    jsonpath.write_text(json.dumps(tweet._json))

In [13]:
IN_PATH = "./twitter-logs/"
OUT_PATH = ""
timestampformat = "EEE MMM dd HH:mm:ss zzzz yyyy"

spark = SparkSession.builder.appName("Twitter-Streaming").getOrCreate()
static= spark.read.json(IN_PATH)
schema = static.limit(100).schema
streaming = spark.readStream.schema(schema).option("maxFilesPerTrigger", 10)\
.json("./twitter-logs/")

In [14]:
static.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- media_url: string (nullable = true)
 |    |    |    |-- media_url_https: string (nullable = true)
 |    |    |    |-- s

In [ ]:
# examining various counts of tweets grouped by user
tweet_counts = streaming.groupBy("user").count()

tweet_query = tweet_counts.writeStream.queryName("tweet_counts")\
.format("memory").outputMode("complete")\
.start()

In [33]:
spark.sql("SELECT * FROM tweet_counts").show()

+--------------------+-----+
|                user|count|
+--------------------+-----+
|{false, Sun Apr 0...|    1|
|{false, Sat Oct 3...|    1|
|{false, Mon Mar 0...|    1|
|{false, Tue Feb 1...|    1|
|{false, Wed Apr 2...|    2|
|{false, Fri Feb 1...|    1|
|{false, Fri Aug 2...|    3|
+--------------------+-----+



In [19]:
selection_query = streaming\
.select("user", "user.favourites_count", "user.followers_count", 
        "user.friends_count", "user.statuses_count")\
.writeStream\
.queryName("tweet_metrics")\
.format("memory")\
.outputMode("append")\
.start()

In [20]:
spark.sql("SELECT * FROM tweet_metrics").show()

+--------------------+----------------+---------------+-------------+--------------+
|                user|favourites_count|followers_count|friends_count|statuses_count|
+--------------------+----------------+---------------+-------------+--------------+
|{false, Sat Oct 3...|             219|         338337|          305|         38742|
|{false, Tue Feb 1...|             525|         804194|         2006|         78242|
|{false, Sun Apr 0...|            1164|       18835049|         1036|        341412|
|{false, Wed Apr 2...|            2929|        8084424|          132|        899825|
|{false, Mon Mar 0...|            3998|         212584|         1389|         69542|
|{false, Wed Apr 2...|            2929|        8084424|          132|        899825|
|{false, Fri Aug 2...|              88|          94766|          130|        198564|
|{false, Fri Aug 2...|              88|          94766|          130|        198564|
|{false, Fri Aug 2...|              88|          94766|          

In [23]:
selection_query = streaming\
.select("user.id", "user.favourites_count", "user.followers_count", 
        "user.friends_count", "user.statuses_count")\
.writeStream\
.queryName("tweet_metrics_agg")\
.format("memory")\
.outputMode("append")\
.start()

In [24]:
spark.sql("SELECT * FROM tweet_metrics_agg").show(truncate= False)

+---------+----------------+---------------+-------------+--------------+
|id       |favourites_count|followers_count|friends_count|statuses_count|
+---------+----------------+---------------+-------------+--------------+
|86478153 |219             |338337         |305          |38742         |
|21114659 |525             |804194         |2006         |78242         |
|3108351  |1164            |18835049       |1036         |341412        |
|36327407 |2929            |8084424        |132          |899825        |
|22545453 |3998            |212584         |1389         |69542         |
|36327407 |2929            |8084424        |132          |899825        |
|180748385|88              |94766          |130          |198564        |
|180748385|88              |94766          |130          |198564        |
|180748385|88              |94766          |130          |198564        |
|20751449 |153             |6979140        |211          |379819        |
|68746721 |7228            |243128    

In [41]:
selection_query = streaming\
.select("user.id", "user.favourites_count", "user.followers_count", 
        "user.friends_count", "user.statuses_count")\
.groupBy("id").count()

In [43]:
selection_query\
.writeStream\
.queryName("tweet_metrics_avg")\
.format("memory")\
.outputMode("complete")\
.start()

In [44]:
spark.sql("SELECT * FROM tweet_metrics_avg").show(truncate= False)

+----------+-----+
|id        |count|
+----------+-----+
|22545453  |1    |
|39922594  |1    |
|37034483  |5    |
|43855487  |3    |
|134758540 |1    |
|22763833  |3    |
|240649814 |1    |
|68746721  |1    |
|3108351   |2    |
|1191912552|2    |
|86478153  |1    |
|180748385 |6    |
|36327407  |2    |
|19897138  |2    |
|21114659  |1    |
|39743812  |1    |
|920488039 |2    |
|20751449  |2    |
|38647512  |2    |
|48008938  |1    |
+----------+-----+



In [25]:
# Examine the streaming queries which are active
spark.streams.active

In [26]:
selection_query = streaming\
.select("user", "retweet_count", "lang", "created_at")\
.writeStream\
.queryName("selection_query_")\
.format("memory")\
.outputMode("append")\
.start()

In [27]:
spark.sql("SELECT * FROM selection_query_").show()

+--------------------+-------------+----+--------------------+
|                user|retweet_count|lang|          created_at|
+--------------------+-------------+----+--------------------+
|{false, Sat Oct 3...|            0|  en|Sun Jun 06 23:50:...|
|{false, Tue Feb 1...|            5|  en|Sun Jun 06 23:39:...|
|{false, Sun Apr 0...|            8|  en|Sun Jun 06 23:45:...|
|{false, Wed Apr 2...|            0|  en|Sun Jun 06 23:40:...|
|{false, Mon Mar 0...|            1|  en|Sun Jun 06 23:39:...|
|{false, Wed Apr 2...|            2|  en|Sun Jun 06 23:50:...|
|{false, Fri Aug 2...|            0|  en|Sun Jun 06 23:45:...|
|{false, Fri Aug 2...|            0|  en|Sun Jun 06 23:50:...|
|{false, Fri Aug 2...|            0|  en|Sun Jun 06 23:40:...|
|{false, Fri Feb 1...|            0|  en|Sun Jun 06 23:50:...|
|{false, Tue Aug 2...|            8|  en|Sun Jun 06 23:27:...|
|{false, Fri May 0...|            3|  en|Sun Jun 06 23:30:...|
|{false, Sun Apr 0...|           12|  en|Sun Jun 06 23:

### Caching

In [ ]:
start= time.time()
sales = spark.read.csv('online-retail-dataset.csv', header= True, inferSchema= True)
sales_country= sales.groupBy("Country").count().collect()
sales_customer= sales.groupBy("CustomerID").count().collect()
sales_invoice= sales.groupBy("InvoiceNo").count().collect()
end= time.time()
print("total time elapsed: ", end-start, "seconds")

In [ ]:
sales.cache()
sales.count()
start= time.time()
sales_country= sales.groupBy("Country").count().collect()
sales_customer= sales.groupBy("CustomerID").count().collect()
sales_invoice= sales.groupBy("InvoiceNo").count().collect()
end= time.time()
print("total time elapsed: ", end-start, "seconds")

In [ ]:
?sales.persist

In [ ]:
from pyspark.storagelevel import StorageLevel
sales.persist(storageLevel= StorageLevel.DISK_ONLY)
sales.count()

### Twitter Default Streaming 

In [ ]:
import json
import tweepy

class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api):
        self.api = api
        self.me = api.me()

    def on_status(self, tweet):
        print(f"{tweet.user.name}:{tweet.text}")

    def on_error(self, status):
        print("Error detected")

# Authenticate to Twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

tweets_listener = MyStreamListener(api)
stream = tweepy.Stream(api.auth, tweets_listener)
stream.filter(track=["Covid-19"], languages=["en"])